In [81]:
import pandas as pd
import numpy as np
from jinja2 import Template

In [82]:
def read_csv_file(file_path):
    """Reads a CSV file and returns a DataFrame."""
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None
    


In [83]:
df = read_csv_file('Berufsrollen_frueher_A.csv')
df = df.replace({np.nan: ''})
def split_semicolon_asterisk(value):
    if pd.isna(value) or value is None or value == '':
        return None
    parts = [p.strip() for p in str(value).split(';') if p.strip() != '']
    processed = []
    for p in parts:
        base = p.split('*')[0].strip().lower()
        if 'berufsrollen' in base:
            base = 'berufsrolle'
        processed.append(base)
    if not processed:
        return None
    if len(processed) == 1:
        return processed[0]
    return processed

df['skos:narrower'] = df['skos:narrower'].apply(split_semicolon_asterisk)
df['skos:broader'] = df['skos:broader'].apply(split_semicolon_asterisk)
df = df.replace({np.nan: None, '' : None})
#berufsrolle = df.to_dict(orient='list')
#concept_scheme = list(df.iloc[0].items())

In [84]:
df = df.iloc[1:] #schmeißt die erste Zeile raus
df = df.drop(columns=["skos:Concept", "uri", "dct:title", "skos:related"])
df = df.T


In [85]:
mapping= {}
i =0
for pref_label in df.iloc[0]:
    i +=1
    newindex =i
    #print (newindex)
    newname =pref_label.split("*")[0].lower()
    #print (pref_label, "->", newname)
    mapping[newindex] = newname

mapping
    

{1: 'ägyptologe',
 2: 'althistoriker',
 3: 'anthropologe',
 4: 'antiquar',
 5: 'archäobotaniker',
 6: 'archäologe',
 7: 'archäozoologe',
 8: 'architekt',
 9: 'archivar',
 10: 'bauarbeiter',
 11: 'bibliothekar',
 12: 'botaniker',
 13: 'diplomat',
 14: 'ethnologe',
 15: 'fotograf',
 16: 'geistliche',
 17: 'geograf',
 18: 'geologe',
 19: 'geophysiker',
 20: 'grabungstechniker',
 21: 'historiker',
 22: 'ingenieur',
 23: 'journalist',
 24: 'jurist',
 25: 'händler',
 26: 'kartograf',
 27: 'kolonialbeamter',
 28: 'kunsthistoriker',
 29: 'künstler',
 30: 'kurator',
 31: 'landesarchäologe',
 32: 'landwirt',
 33: 'lehrer',
 34: 'linguist',
 35: 'mathematiker',
 36: 'mediziner',
 37: 'militärangehörige',
 38: 'museumsleiter',
 39: 'offizier',
 40: 'paläontologe',
 41: 'philologe',
 42: 'politiker',
 43: 'restaurator',
 44: 'schriftsteller',
 45: 'soldat',
 46: 'staatsoberhaupt',
 47: 'theolog',
 48: 'unternehmer',
 49: 'vermessungsingenieur',
 50: 'grafiker'}

In [86]:
df1=df.rename(columns=mapping)

In [87]:
berufsrollen = df1.to_dict()
berufsrollen

{'ägyptologe': {'skos:prefLabel': 'Ägyptologe*in',
  'skos:altLabel': 'Ägyptenforscher*in',
  'dct:description': 'Wissenschaftler*in, der/die das alte Ägypten erforscht.',
  'skos:definition': 'Person, die sich mit der Erforschung der Geschichte, Sprache, Kultur und materiellen Hinterlassenschaften des alten Ägypten beschäftigt.',
  'skos:scopeNote': 'Die Ägpytologie hat sich vor der Vor- und Frühgeschichte als eigenes Fach etabliert.',
  'skos:broader': 'berufsrolle',
  'skos:narrower': None},
 'althistoriker': {'skos:prefLabel': 'Althistoriker*in',
  'skos:altLabel': None,
  'dct:description': 'Historiker*in, der/die sich auf die Alte Geschichte (griechisch-römische Antike) spezialisiert hat.',
  'skos:definition': 'Wissenschaftler*in, die sich mit der politischen, sozialen und kulturellen Geschichte der griechisch-römischen Antike beschäftigt.',
  'skos:scopeNote': 'Althistoriker*innen haben sich außer mit Texten oft auch mit materiellen Hinterlassenschaften beschäftigt.',
  'skos:b

In [ ]:
with open('arch_template.j2') as f:
    tmpl = Template(f.read())

archt_ttl =tmpl.render(trim_blocks=True, lstrip_blocks=True, concepts=berufsrollen, statics = [
    "skos:prefLabel",
    "dct:description",
    "skos:altLabel",
    "skos:definition",
    "skos:scopeNote"])
with open('../arch_leiza.ttl', 'w') as f:
    f.write(archt_ttl)